# Broker Actuals Check

In [1]:
import numpy as np
import pandas as pd
import os

import bql
bq = bql.Service()

from IPython.display import display
from datetime import datetime
import json

from src import fetch_csmg_deep_data, parse_individual_bucket_main
from bloomberg.gdto.bcos.bcos_client import BCOSClient, BCOS_PROD_URL, CORP_TO_PROD_PROXIES
from src.utils import credentials

ModuleNotFoundError: No module named 'bloomberg.gdto'

In [2]:
import os
os.getcwd().replace('\\', '/')

'C:/Users/oghazizadeh/AppData/Local/bipy/30789296/projects/e3291812464348a4ac259fdf49f0ecbd'

In [2]:
tot_bcos_client = BCOSClient(
    api_url=BCOS_PROD_URL,
    api_user=credentials['API_ACCOUNT'],
    api_key=credentials['API_KEY'],
    proxy=CORP_TO_PROD_PROXIES
)
tot_bucket = credentials['bucket']

In [3]:
ticker = 'AAPL US Equity'
calcrt = 'IS900'
segment_id = None

In [4]:
csmg_deep_data = fetch_csmg_deep_data(ticker=ticker, bucket=tot_bucket, client=tot_bcos_client)
broker_actuals = parse_individual_bucket_main(ticker, csmg_deep_data, calcrt=calcrt, segment_id=segment_id)

In [5]:
broker_actuals.period = broker_actuals.period.apply(lambda x: f"{x[:4]} {x[4:]}")
queries = broker_actuals.apply(lambda x: f"get({x.calcrt}(fpt={'A' if 'A' in x.period else 'Q'}, fpr='{x.period}')) for('{x.segment_id}') preferences(dropcols=[REVISION_DATE, AS_OF_DATE, CURRENCY], addcols=[period])", 1).pipe(list)
response = bq.execute_many(queries)
financials = (
    pd.concat([bql.combined_df(res) for res in response])
        .reset_index()
        .melt(id_vars=['ID', 'PERIOD_END_DATE', 'PERIOD'])
        .dropna(subset=['value'])
        .reset_index(drop=True)
        .pipe(lambda x: x.set_axis(x.columns.str.lower(), axis='columns'))
)

### NOTE
Next step is to join financials and broker actuals for comparison
<br>
This code needs to be refactored if it is turned into a routine check

In [6]:
broker_actuals.head()

,calcrt,segment_id,period,broker_actuals
0,IS900,AAPL US Equity,1994 A,0.000873
1,IS900,AAPL US Equity,1994 Q1,0.003052
2,IS900,AAPL US Equity,1994 Q2,0.001276
3,IS900,AAPL US Equity,1994 Q3,0.010442
4,IS900,AAPL US Equity,1994 Q4,0.008454


In [7]:
financials.head()

,id,period_end_date,period,variable,value
0,AAPL US Equity,1994-09-30,1994 A,"IS900(fpt=A,fpr='1994 A')",0.023304
1,AAPL US Equity,1993-12-31,1994 Q1,"IS900(fpt=Q,fpr='1994 Q1')",0.003036
2,AAPL US Equity,1994-03-31,1994 Q2,"IS900(fpt=Q,fpr='1994 Q2')",0.001339
3,AAPL US Equity,1994-07-01,1994 Q3,"IS900(fpt=Q,fpr='1994 Q3')",0.010357
4,AAPL US Equity,1994-09-30,1994 Q4,"IS900(fpt=Q,fpr='1994 Q4')",0.008482
